In [40]:
# import libraries needed for the project 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [41]:
# open the data file

data = pd.read_csv('./data/AirQualityUCI.csv', delimiter=';')


## Data explorations

In [42]:
data.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578",NaN,NaN
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255",NaN,NaN
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502",NaN,NaN
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867",NaN,NaN
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888",NaN,NaN


In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9471 entries, 0 to 9470
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           9357 non-null   object 
 1   Time           9357 non-null   object 
 2   CO(GT)         9357 non-null   object 
 3   PT08.S1(CO)    9357 non-null   float64
 4   NMHC(GT)       9357 non-null   float64
 5   C6H6(GT)       9357 non-null   object 
 6   PT08.S2(NMHC)  9357 non-null   float64
 7   NOx(GT)        9357 non-null   float64
 8   PT08.S3(NOx)   9357 non-null   float64
 9   NO2(GT)        9357 non-null   float64
 10  PT08.S4(NO2)   9357 non-null   float64
 11  PT08.S5(O3)    9357 non-null   float64
 12  T              9357 non-null   object 
 13  RH             9357 non-null   object 
 14  AH             9357 non-null   object 
 15  Unnamed: 15    0 non-null      float64
 16  Unnamed: 16    0 non-null      float64
dtypes: float64(10), object(7)
memory usage: 1.2+ MB


Let's deal with NaN data. From the data above, we know that there are 3842 rows with NaN values, which represents 17.7% of the total values. Since this correspond to a small percentage, we proceed to delete the rows that contain Nan values.


In [44]:
data = data.drop(['Unnamed: 15'], axis=1)
data = data.drop(['Unnamed: 16'], axis=1)

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9471 entries, 0 to 9470
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           9357 non-null   object 
 1   Time           9357 non-null   object 
 2   CO(GT)         9357 non-null   object 
 3   PT08.S1(CO)    9357 non-null   float64
 4   NMHC(GT)       9357 non-null   float64
 5   C6H6(GT)       9357 non-null   object 
 6   PT08.S2(NMHC)  9357 non-null   float64
 7   NOx(GT)        9357 non-null   float64
 8   PT08.S3(NOx)   9357 non-null   float64
 9   NO2(GT)        9357 non-null   float64
 10  PT08.S4(NO2)   9357 non-null   float64
 11  PT08.S5(O3)    9357 non-null   float64
 12  T              9357 non-null   object 
 13  RH             9357 non-null   object 
 14  AH             9357 non-null   object 
dtypes: float64(8), object(7)
memory usage: 1.1+ MB


In [45]:
data = data.dropna()


In [46]:
#data['CO(GT)'].tail()
data.sort_values('CO(GT)').tail()
#GT means gygatone?

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
5690,02/11/2004,20.00.00,"9,2",1778.0,-200.0,"48,2",1935.0,859.0,349.0,119.0,2643.0,1927.0,"20,9","67,5","1,6539"
6696,14/12/2004,18.00.00,"9,3",-200.0,-200.0,"-200,0",-200.0,1310.0,-200.0,248.0,-200.0,-200.0,-200,-200,-200
6409,02/12/2004,19.00.00,"9,4",1816.0,-200.0,"43,9",1851.0,1184.0,341.0,171.0,2405.0,2069.0,"15,4","73,0","1,2688"
5520,26/10/2004,18.00.00,"9,5",1908.0,-200.0,"52,1",2007.0,952.0,325.0,180.0,2775.0,2372.0,"22,5","61,5","1,6579"
6672,13/12/2004,18.00.00,"9,9",1881.0,-200.0,"50,8",1983.0,1479.0,334.0,269.0,2271.0,2523.0,"12,6","55,9","0,8142"


In [47]:
len(data[data['CO(GT)'] == '-200'])


1592

In [48]:
data['CO(GT)'].value_counts()

-200    1592
1,4      279
1,6      275
1,5      273
1,1      262
        ... 
10,1       1
9,3        1
7          1
9,2        1
9,9        1
Name: CO(GT), Length: 104, dtype: int64

In [49]:
data[(data != -200).all(1)]


,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578"
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255"
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502"
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867"
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,30/04/2004,20.00.00,"4,4",1449.0,501.0,"19,5",1282.0,254.0,625.0,133.0,2100.0,1569.0,"19,1","61,1","1,3345"
1227,30/04/2004,21.00.00,"3,1",1363.0,234.0,"15,1",1152.0,189.0,684.0,110.0,1951.0,1495.0,"18,2","65,4","1,3529"
1228,30/04/2004,22.00.00,3,1371.0,212.0,"14,6",1136.0,174.0,689.0,102.0,1927.0,1471.0,"18,1","66,1","1,3579"
1229,30/04/2004,23.00.00,"3,1",1406.0,275.0,"13,7",1107.0,167.0,718.0,108.0,1872.0,1384.0,"17,7","66,9","1,3422"


In [50]:
data

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578"
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255"
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502"
3,10/03/2004,21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867"
4,10/03/2004,22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,04/04/2005,10.00.00,"3,1",1314.0,-200.0,"13,5",1101.0,472.0,539.0,190.0,1374.0,1729.0,"21,9","29,3","0,7568"
9353,04/04/2005,11.00.00,"2,4",1163.0,-200.0,"11,4",1027.0,353.0,604.0,179.0,1264.0,1269.0,"24,3","23,7","0,7119"
9354,04/04/2005,12.00.00,"2,4",1142.0,-200.0,"12,4",1063.0,293.0,603.0,175.0,1241.0,1092.0,"26,9","18,3","0,6406"
9355,04/04/2005,13.00.00,"2,1",1003.0,-200.0,"9,5",961.0,235.0,702.0,156.0,1041.0,770.0,"28,3","13,5","0,5139"


In [56]:
data['CO(GT)'] = data['CO(GT)'].replace(","," ")
data['CO(GT)']

0       2,6
1         2
2       2,2
3       2,2
4       1,6
       ... 
9352    3,1
9353    2.4
9354    2.4
9355    2,1
9356    2,2
Name: CO(GT), Length: 9357, dtype: object

In [62]:
data['CO(GT)'] = data['CO(GT)'].astype(str)
data['CO(GT)'] = data['CO(GT)'].str.replace(",",".")
data['CO(GT)']

0       2.6
1         2
2       2.2
3       2.2
4       1.6
       ... 
9352    3.1
9353    2.4
9354    2.4
9355    2.1
9356    2.2
Name: CO(GT), Length: 9357, dtype: object

In [64]:
data['C6H6(GT)'] = data['C6H6(GT)'].str.replace(",",".")
data['C6H6(GT)']

0       11.9
1        9.4
2        9.0
3        9.2
4        6.5
        ... 
9352    13.5
9353    11.4
9354    12.4
9355     9.5
9356    11.9
Name: C6H6(GT), Length: 9357, dtype: object

In [65]:
data['T'] = data['T'].str.replace(",",".")
data['T']

0       13.6
1       13.3
2       11.9
3       11.0
4       11.2
        ... 
9352    21.9
9353    24.3
9354    26.9
9355    28.3
9356    28.5
Name: T, Length: 9357, dtype: object

In [66]:
data['RH'] = data['RH'].str.replace(",",".")
data['RH']

0       48.9
1       47.7
2       54.0
3       60.0
4       59.6
        ... 
9352    29.3
9353    23.7
9354    18.3
9355    13.5
9356    13.1
Name: RH, Length: 9357, dtype: object

In [67]:
data['AH'] = data['AH'].str.replace(",",".")
data['AH']

0       0.7578
1       0.7255
2       0.7502
3       0.7867
4       0.7888
         ...  
9352    0.7568
9353    0.7119
9354    0.6406
9355    0.5139
9356    0.5028
Name: AH, Length: 9357, dtype: object

From this first screening, we can see that sqft_basement is an object, so we convert it into float. But when we try to apply the function astype, it gives an error. That's why we have to epxlore the column a little further. By reading the corresponding error of astype and using the function .value_counts() we can see that the problem is that there is the sign "?". 


In [ ]:

data['sqft_basement'].value_counts()

To proceed with our analysis, we detete the rows that contain "?" 

In [ ]:
data = data[(data.sqft_basement != '?' )]
data.info()

We can now proceed to convert sqft_basement into a float type

In [ ]:
data['sqft_basement'] = data['sqft_basement'].astype(str).astype(float)

In [ ]:
data.info()

Above we have confirm that the sqft_basement is now a float.

## CATEGORICAL VS CONTINUOUS DATA ANALYSIS

Now let's identify which variables are categorical and which ones are continuous. We will do it by plotting the data. From the graphs below, we can determine that waterfront, view, condition, grade, years built, renovated year, number of bedrooms, numbero of bathrooms and number of floors are categorical variables. 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

 # crate the figure and axes
fig, axes = plt.subplots(7, 3, figsize=(24, 30))   

# unpack all the axes subplots
axe = axes.ravel()

# assign the plot to each subplot in axe
for i, c in enumerate(data.columns):
    
    data.plot(kind='scatter', x=c, y='price', ax=axe[i], alpha=0.4, color='b')

We conver the "years_built" variable into a continous variable by calculating the years since it was renovated.

In [ ]:
data['years_built'] = 2022 - data['yr_built']
data.info()


## EFFECT OF RENOVATION ON PRICE 

We start our analysis determining the effect of renovation in house prices. Let's explore the data a bit further.

In [ ]:
data.yr_renovated.value_counts()

From the data, we can observed that some years of renovation correspond to 0, which means that these houses have not been renovated. 
What we are going to do, is to create 2 grups, one of houses that have not been renovated and the other one of renovated houses. After that, we will perform a t-test to determine whether there is a significant change between these two groups. 


Hypothesis:
H0 = there are no differences between renovated and non-renovated houses
Ha = there are differences between renovated and non-renovated houses


In [ ]:
no_renovated = data[data.yr_renovated == 0]['price']
renovated = data[data.yr_renovated != 0]['price']

# Calculate the 2-sided p-value for a t-test comparing the renovated vs no-renovated groups
from scipy import stats

stats.ttest_ind(no_renovated, renovated, equal_var=False)[1]


The above result indicates that there is a signfincant difference between renovated and non-renovated houeses. Which one is more expensive? Let's see:

In [ ]:
print(no_renovated.mean(), renovated.mean())

The renovated houses are more expensive. How much?  1.4-fold more expensive, as seen below. 

In [ ]:
fold_increase = (renovated.mean()/no_renovated.mean())
fold_increase

Based on this data, we select only the non-renovated houses, as our main is to find the cheapest houses for the WHPO.

In [ ]:
data_new = data.drop(data[data.yr_renovated != 0].index)
data_new.yr_renovated.value_counts() #to confirm that we only selected the non-renovated

In [ ]:
data_new.info()

## EFFECTS OF WATERVIEW ON PRICE

To study the effect of waterviews on price, we performed the same analysis that with renovation: a t-test to compare houses with and without waterview 

In [ ]:
no_water_view = data_new[data_new.waterfront == 0]['price']
water_view = data_new[data_new.waterfront != 0]['price']

# Calculate the 2-sided p-value for a t-test comparing the renovated vs no-renovated groups
from scipy import stats

stats.ttest_ind(water_view, no_water_view, equal_var=False)[1]

The t-test analysis confirm that there is a statistically signifcant differences between houses with and whithout waterview. 
Below we can see the average price for both groups.

In [ ]:
print(water_view.mean(), no_water_view.mean())

Based on the data above, we select only the houses that do not have waterviews for our further analysis

In [ ]:
data_new = data_new.drop(data_new[data_new.waterfront != 0].index)
data_new

In [ ]:
data_new.waterfront.value_counts() #to confirm that we only have the houses without waterviews

In [ ]:
data_new.info()

## EFFECTS OF ZIPCODES ON PRICE

We want now to determine which are the zipcodes that are most expensive. Since zipcodes is a categorical variable, we will use get_dummies to work with tat.

In [ ]:
zipcode_dummies = pd.get_dummies(data_new['zipcode'], prefix='zip', drop_first=True)

In [ ]:
zipcode_analysis = data.drop(['zipcode'], axis=1)
zipcode_analysis = pd.concat([data_new['price'], zipcode_dummies], axis=1)
zipcode_analysis.head()




In [ ]:
X_zip = zipcode_analysis.drop('price', axis=1)
y_zip = zipcode_analysis['price']

import statsmodels.api as sm
X_zip_int = sm.add_constant(X_zip)
model = sm.OLS(y_zip,X_zip_int).fit()
model.summary()

The adjusted R-squared value for the whole mode is 0.457, which is a very low value. This means that only 45% of the price variability can be explained by zipcode. However, in this analysis we want to focus on particular zipcodes. 

From this analysis we can see that several zipocdes positively and signifcantly increase the price (coefient positive and p < 0.05), and some that do not. We also observe that even some of them have a negative correlation with price, they are not significant. 

We want to identify which ones are the ones that positively and signficnatly increase the price and which ones do not. 

In [ ]:
columns = ['index','pvalue']

pvalue_positive_table = pd.DataFrame(columns=columns)
pvalue_negative_table = pd.DataFrame(columns=columns)


for i, a in enumerate(model.pvalues):
    if a < 0.05:
        pvalue_positive_table.loc[i] = [i,a]
    else:
        pvalue_negative_table.loc[i] = [i,a]




In [ ]:
pvalue_positive_table

In [ ]:
pvalue_negative_table

In [ ]:
index = pvalue_negative_table.index
index

In [ ]:
zipcodes_of_interstet = []

for i in index:
    zipcodes_of_interstet.append(zipcode_analysis.columns[i])

zipcodes_of_interstet_name = []



Below, we can see the zipcodes of interset, which means, the zipcodes that do not signficnatly incrase the house prices. Herein, these zipcodes are the ones that will have the lowest house prices

In [ ]:
for i in zipcodes_of_interstet:
     zipcodes_of_interstet_name.append(i[-5:])

zipcodes_of_interstet_name

In [ ]:
index_NO_interest = pvalue_positive_table.index


zipcodes_of_NO_interstet = []

for i in index_NO_interest:
    zipcodes_of_NO_interstet.append(zipcode_analysis.columns[i])

zipcodes_of_NO_interstet_name = []

for i in zipcodes_of_NO_interstet:
     zipcodes_of_NO_interstet_name.append(i[-5:])

zipcodes_of_NO_interstet_name

In [ ]:
#just to refresh our data:

data_new

In [ ]:
#as mentioend above, now we want to select only the houses corresponding to our zipcodes of interest

data_new2 =  data_new[data_new['zipcode'].isin(zipcodes_of_interstet_name)]
data_new2.zipcode.value_counts()

In [ ]:
data_new2.info()

## EFFECTS OF CONDITION ON PRICE

Condition is also a categorical variable, and we will apply get_dummies to deal with it. 

In [ ]:
condition_dummies = pd.get_dummies(data_new2['condition'], prefix='cond', drop_first=True)

In [ ]:
condition_analysis = data_new2.drop(['condition'], axis=1)
condition_analysis = pd.concat([data_new2['price'], condition_dummies], axis=1)
condition_analysis.head(10)


In [ ]:
X_cond = condition_analysis.drop('price', axis=1)
y_cond = condition_analysis['price']

X_cond_int = sm.add_constant(X_cond)
model_cond = sm.OLS(y_cond,X_cond_int).fit()
model_cond.summary()

The adjusted R2-value is very low (0.015). It means that only 1.5% of the price variablity can be explained by this model. However, we have performed this analysis to focus on the value of each condition individually. 

Similar to zipcodes, there are some conditoins that signicnatly and positively increase the house price. We perform the same analysis to udnerstand which conditions impact the price and which ones do not. 

In [ ]:
columns = ['index','pvalue']

condition_pvalue_positive_table = pd.DataFrame(columns=columns)
condition_pvalue_negative_table = pd.DataFrame(columns=columns)


for i, a in enumerate(model_cond.pvalues):
    if a < 0.05:
        condition_pvalue_positive_table.loc[i] = [i,a]
    else:
        condition_pvalue_negative_table.loc[i] = [i,a]


condition_pvalue_positive_table

In [ ]:
condition_pvalue_negative_table

In [ ]:
cond_grade = condition_pvalue_negative_table.index
cond_grade

In [ ]:
cond_of_interstet = []

for i in cond_grade:
    cond_of_interstet.append(condition_analysis.columns[i])

cond_of_interstet

In [ ]:
cond_grade_negative = condition_pvalue_positive_table.index
cond_grade_negative

cond_no_interstet = []

for i in cond_grade_negative:
    cond_no_interstet.append(condition_analysis.columns[i])

cond_no_interstet

In [ ]:
data_new2.condition.value_counts()


From all the analysis above, we can conclude that condition 3 and above signficnatly increase the price of the houses. However, we can't filter our data only based on condition 1 and 2, because this only correspond to 27 datapoints. We will proceed with the analysis without this filter, but this is an insight that we will give to our clients.

## EFFECTS OF GRADE ON PRICE

Grade is also a categorical variable, and we will apply get_dummies to deal with it. 

In [ ]:
grade_dummies = pd.get_dummies(data_new2['grade'], prefix='grd', drop_first=True)

grade_analysis = data_new2.drop(['grade'], axis=1)
grade_analysis = pd.concat([data_new2['price'], grade_dummies], axis=1)
grade_analysis.head(10)

In [ ]:
X_grade = grade_analysis.drop('price', axis=1)
y_grade = grade_analysis['price']

X_grade_int = sm.add_constant(X_grade)
model_grade = sm.OLS(y_grade,X_grade_int).fit()
model_grade.summary()

Similar to condition, the adjusted R2-value is very low (0.433). This means that only 43.3% of the price variability can be explained by this model. However, we have performed this analysis to focus on the value of each grade indivually. 

Similar to grade, there are some grades that signficantly and positively increase the house price. We perform the same analysis to understand which grades impact the price and which ones do not. 

In [ ]:
columns = ['index','pvalue']

grade_pvalue_positive_table = pd.DataFrame(columns=columns)
grade_pvalue_negative_table = pd.DataFrame(columns=columns)


for i, a in enumerate(model_grade.pvalues):
    if a < 0.05:
        grade_pvalue_positive_table.loc[i] = [i,a]
    else:
        grade_pvalue_negative_table.loc[i] = [i,a]


grade_pvalue_positive_table

In [ ]:
grade_pvalue_negative_table

In [ ]:
index_grade = grade_pvalue_negative_table.index
index_grade

In [ ]:
grade_analysis

In [ ]:
grade_of_interstet = []

for i in index_grade:
    grade_of_interstet.append(grade_analysis.columns[i])

grade_of_interstet

In [ ]:
index_grade_pos = grade_pvalue_positive_table.index

grade_no_interstet = []

for i in index_grade_pos:
    grade_no_interstet.append(grade_analysis.columns[i])

grade_no_interstet

From the analysis above, we can determine that grade 9 and above significantly increase the house price, and therefore we  select only the houses with grade bewlow 9, as they are the cheaper ones. 

In [ ]:
     
grade_of_interstet_name = []

for i in grade_of_interstet:
     grade_of_interstet_name.append(i[-1:])

grade_of_interstet_name

In [ ]:
# we select houses below grade 9
data_new3 =  data_new2[data_new2['grade'].isin(grade_of_interstet_name)]

data_new3.grade.value_counts()

In [ ]:
data_new3.info()

## EFFECTS OF NUMBER OF BEDROOMS ON PRICE


In [ ]:
bedrooms_dummies = pd.get_dummies(data_new3['bedrooms'], prefix='bdr', drop_first=True)

bedrooms_analysis = data_new3.drop(['grade'], axis=1)
bedrooms_analysis = pd.concat([data_new3['price'], bedrooms_dummies], axis=1)
bedrooms_analysis.head(10)

In [ ]:
X_bed = bedrooms_analysis.drop('price', axis=1)
y_bed = bedrooms_analysis['price']

X_bed_int = sm.add_constant(X_bed)
model_beds = sm.OLS(y_bed,X_bed_int).fit()
model_beds.summary()

The adjusted R-squared value for the whole model is 0.132, which is a very low value. This means that only 13.2% of the price variability can be explained by this model.

In [ ]:
columns = ['index','pvalue']

bedrooms_pvalue_positive_table = pd.DataFrame(columns=columns)
bedrooms_pvalue_negative_table = pd.DataFrame(columns=columns)


for i, a in enumerate(model_beds.pvalues):
    if a < 0.05:
        bedrooms_pvalue_positive_table.loc[i] = [i,a]
    else:
        bedrooms_pvalue_negative_table.loc[i] = [i,a]


bedrooms_pvalue_positive_table

In [ ]:
bedrooms_pvalue_negative_table

What this analysis indicate, is that the number of bedrooms always have an impact on the price. And the more number of rooms, the more expensive the price (since the coeficient increases with number of rooms).
It will be up to WHPO to decide the number of rooms they want for their houses. Maybe they want one room for single people, or 3 for families.... but they need to be aware that the bigger the number of rooms, the more expensive the houses.
Considering that WHPO may be intersted in different houses sizes, we won't furhter narrow our subset of intersting houses. 

Now that we have selected the houses with the lowest price, we want to determine how several continuous variables affect the price of the selected houses. 


## EFFECTS OF YEARS SINCE BUILT, SQFT LIVING AREA AND SQFT LIVING LOT ON PRICE 

In [ ]:
data_new3.columns

In [ ]:
#We focus on the following variables, which are continuous variables:

continuous_norm = ['price','sqft_living','sqft_lot', 'years_built']


Let's see if these variables follow a normal distribution

In [ ]:

import warnings
warnings.filterwarnings('ignore')
fig = plt.figure(figsize = (10,10))
ax = fig.gca()
data_new3[continuous_norm].hist(ax = ax);

We try to improve the distribution by logaritmic transformation and normalization

In [ ]:


# Log transform and normalize
data_cont = data_new3[continuous_norm]

# log features
log_names = [f'{column}_log' for column in data_cont.columns]
data_log = np.log(data_cont)
data_log.columns = log_names
# normalize (subract mean and divide by std)
def normalize(feature):
    return (feature - feature.mean()) / feature.std()

data_log_norm = data_log.apply(normalize)

data_log_norm.info()

Let's visualize the changes in the distribution:

In [ ]:
warnings.filterwarnings('ignore')
fig = plt.figure(figsize = (10,10))
ax = fig.gca()
data_log_norm.hist(ax = ax);

In [ ]:
#let's confirm the data that we have

data_log_norm

Now we create a regression model with these variables:

In [ ]:
X_new = data_log_norm.drop('price_log', axis=1)
y_new = data_log_norm['price_log']

import statsmodels.api as sm
X_new_int = sm.add_constant(X_new)
model_new = sm.OLS(y_new,X_new_int).fit()
model_new.summary()


The adjusted R-squared value for the whole mode is 0.446, which is a  low value. This means that only 44.6% of the price variability can be explained by this model.
To try to improve our model, let's check the colinearility of the variables:

In [ ]:
data_log_norm.corr()
abs(data_log_norm.corr()) > 0.75

# save absolute value of correlation matrix as a data frame
# converts all values to absolute value
# stacks the row:column pairs into a multindex
# reset the index to set the multindex to seperate columns
# sort values. 0 is the column automatically generated by the stacking
df=data_log_norm.corr().abs().stack().reset_index().sort_values(0, ascending=False)
# zip the variable name columns (Which were only named level_0 and level_1 by default) in a new column named "pairs"
df['pairs'] = list(zip(df.level_0, df.level_1))
# set index to pairs
df.set_index(['pairs'], inplace = True)
#d rop level columns
df.drop(columns=['level_1', 'level_0'], inplace = True)
# rename correlation column as cc rather than 0
df.columns = ['cc']
# drop duplicates. This could be dangerous if you have variables perfectly correlated with variables other than themselves.
# for the sake of exercise, kept it in.
df.drop_duplicates(inplace=True)

df[(df.cc>.75) & (df.cc <1)]

From the analysis above, it doesn't seem to be any considerable correlation between variables. Now let's study interaction between variables

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

regression = LinearRegression()

crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)
baseline = np.mean(cross_val_score(regression, X_new, y_new, scoring='r2', cv=crossvalidation))
baseline

In [ ]:
from itertools import combinations

crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)

interactions = []

feat_combinations = combinations(X_new.columns, 2)

data = X_new.copy()
for i, (a, b) in enumerate(feat_combinations):
    data['interaction'] = data[a] * data[b]
    score = np.mean(cross_val_score(regression, data, y_new, scoring='r2', cv=crossvalidation))
    if score > baseline:
        interactions.append((a, b, round(score,3)))
    
    if i % 50 == 0:
        print(i)
            
print('Top interaction: %s' %sorted(interactions, key=lambda inter: inter[2], reverse=True)[:1])

So from the analysis above, there are not interactions or colinearitly that we should we aware and adjust our model based on. As such, let's refresh our model and gain some insights: 

In [ ]:
X_new = data_log_norm.drop('price_log', axis=1)
y_new = data_log_norm['price_log']

import statsmodels.api as sm
X_new_int = sm.add_constant(X_new)
model_new = sm.OLS(y_new,X_new_int).fit()
model_new.summary()

SQFT living area and lot have a significant positive contribution to the house price in our subset of interest (the more area, the more expensive).
Years since built has a significant negative contribution to the price (the oldest, the cheapest).
sqft living area is the variable that affects the most to the price, as it has the highest coefficient value (coef = 0.56) and a p_value < 0.05.


## MODEL VALIDATION

Finally we validate our model creating a test sample with 20% of the datapoints. 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

print(len(X_train), len(X_test), len(y_train), len(y_test))

from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

from sklearn.metrics import mean_squared_error
train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train Mean Squarred Error:', train_mse)
print('Test Mean Squarred Error:', test_mse)

The fact that there are no big difference between Test mean squarred error and Train mean squarred error validates our model. 

## CONCLUSIONS
We provide the following business recomendations:

* **Recommendations 1**: Look for non-renovated houses.
* **Recommendations 2**: Look for houses that have no waterview.
* **Recommendations 3**: Look for houses in the following zipcodes: 98002, 98003, 98022, 98023, 98030, 98031, 98032, 98055, 98106, 98148, 98168, 98178, 98188, 98198.
* **Recommendations 4**: Look for houses below condition 3.
* **Recommendations 5**: Look for houses below grade 9.
* **Recommendations 6**: center your search on houses with the lowest sqft living. Older houses will be cheaper